In [1]:
%load_ext autoreload
%autoreload 2
from utils import feature_eng, visualization
# plot the reward function with plotly as line plot
import plotly.io as pio
import numpy as np
import plotly.graph_objs as go
import pmdarima as pm
from pmdarima.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import pandas as pd

from utils import eval

In [8]:
pio.renderers.default = 'iframe'

measurements = feature_eng.get_data_df("train", "B")
fig = visualization.plot_substation_measurements(measurements, "B")
fig.show()

In [9]:
fig = visualization.plot_production(measurements, ['ZLQZLQSPHSDL_VNUUIST'])
fig.show()

In [11]:
is_data = measurements['ZLQZLQSPHSDL_VNUUIST']
target_data = measurements['ZLQZLQSPHSDL_VNUUSOL']
shunt_reactor = measurements['ZLQZLQ110KOMDROQ']

cut_off = 4 * 24 * 8
reactor_border = 20

train, test = train_test_split(is_data[:-cut_off].values, train_size=len(is_data[:-cut_off])-1)

model = pm.ARIMA(order=(2, 1, 2), seasonal_order=(1, 0, 1, 4))
print("fertig")


fertig


In [46]:
# measurement points are separated by 15 minutes. *4 to get to one hour.
time = 4 * 4
loops = 2

predictions = []
for i in range(loops):
    train, test = train_test_split(is_data[:-cut_off-i].values, train_size=len(is_data[:-cut_off-i])-1)
    print(len(train))
    print(len(pd.DataFrame(shunt_reactor[:-cut_off-i-1])))
    model_fit = model.fit(y=train, X=pd.DataFrame(shunt_reactor[:-cut_off-i-1]))
    preds = model_fit.predict(time, pd.DataFrame(shunt_reactor[0:16]))
    predictions.append(preds)
print("fertig")

34172
34172


C:\_dev\edh2023\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



34171
34171


C:\_dev\edh2023\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



fertig


In [48]:
print(len(is_data))
print(len(preds))

predicted_datas = []

for i in range(loops):
    last_element = is_data.index[len(is_data)-cut_off-i]
    predicted_data = pd.Series(predictions[i], index=pd.date_range(last_element, periods=(time), freq='15T'))
    predicted_datas.append(predicted_data)

print(is_data)
print(predicted_data)

is_reactor_running = shunt_reactor[-1] > 20

scatters = []
scatters.append(go.Scatter(x=is_data.index, y=is_data, name="IST"))

for i in range(loops):
    scatters.append(go.Scatter(x=predicted_datas[i].index, y=predicted_datas[i], name="PREDICT"+str(i)))

fig = go.Figure(data=scatters)

# Update the layout
fig.update_layout(title="Nice", xaxis_title='Date', yaxis_title='kV')
fig.show()

34941
16
2022-01-01 01:00:00    239.816895
2022-01-01 01:15:00    239.816895
2022-01-01 01:30:00    239.868171
2022-01-01 01:45:00    240.080566
2022-01-01 02:00:00    240.080566
                          ...    
2022-12-30 23:00:00    239.553207
2022-12-30 23:15:00    240.323946
2022-12-30 23:30:00    240.476089
2022-12-30 23:45:00    240.476089
2022-12-31 00:00:00    240.476089
Freq: 15T, Name: ZLQZLQSPHSDL_VNUUIST, Length: 34941, dtype: float64
2022-12-23 00:00:00    240.324815
2022-12-23 00:15:00    240.262650
2022-12-23 00:30:00    240.206112
2022-12-23 00:45:00    240.151141
2022-12-23 01:00:00    240.103284
2022-12-23 01:15:00    240.059497
2022-12-23 01:30:00    240.020513
2022-12-23 01:45:00    239.985438
2022-12-23 02:00:00    239.953905
2022-12-23 02:15:00    239.925554
2022-12-23 02:30:00    239.900056
2022-12-23 02:45:00    239.877127
2022-12-23 03:00:00    239.856508
2022-12-23 03:15:00    239.837967
2022-12-23 03:30:00    239.821293
2022-12-23 03:45:00           NaN
Freq